In [ ]:
%reload_ext autoreload
%autoreload 2

from pytorch_lightning import seed_everything
from src.acnets.deep import WaveletModel, MaskedModel
from src.acnets.datasets import LEMONDataModule, Julia2018DataModule

for n_embeddings in [32]:

    seed_everything(42)
    input_type = 'tconn_networks'

    pretrain_datamodule = LEMONDataModule(
        atlas='dosenbach2010', kind='partial correlation',
        aggregation_strategy=input_type,
        test_ratio=.1, val_ratio=.05,
        segment_length=124,
        n_subjects=215, batch_size=32, shuffle=True)
    pretrain_datamodule.setup()

    finetune_datamodule = Julia2018DataModule(
        atlas='dosenbach2010', kind='partial correlation',
        aggregation_strategy=input_type,
        segment_length=124,
        test_ratio=.25, batch_size=8, shuffle=True)
    finetune_datamodule.setup()

    n_features = finetune_datamodule.train[0][0].shape[-1]

    model = MaskedModel(n_features * n_features,
                        n_embeddings,
                        input_type=input_type)

    trainer = model.fit(pretrain_datamodule, max_epochs=100, phase='pretrain')
    tuner = model.fit(finetune_datamodule, max_epochs=10000, phase='finetune')